In [1]:
import os

In [2]:
hyper_jaccard_path = "../bindings/python/src/hyper_jaccard.rs"
hyper_sketches_path = "../bindings/python/src/hyper_sketching.rs"
precision = (4, 16)
bits = (4, 6)
hops = (2, 10)

In [3]:
file_lines = []

with open(hyper_jaccard_path, "r") as f:
    for line in f:
        if "{python_generated}" in line:
            continue
        file_lines.append(line)
        if "{python_macro}" in line:
            if "{bits}" not in line or "{precision}" not in line:
                raise ValueError(
                    f"We detected a python macro line, namely {line}, "
                    "but it does not contain the expected tags {bits} and {precision}."
                )
            # We split the line at the comments
            spacer = "//"
            if "///" in line:
                spacer = "///"
            initial_spaces, python_macro = line.split(spacer)
            for p in range(precision[0], precision[1] + 1):
                for b in range(bits[0], bits[1] + 1):
                    populated_line = initial_spaces + python_macro.strip().format(
                        bits=b,
                        precision=p,
                        python_macro="// {python_generated}\n"
                    )
                    file_lines.append(populated_line)
            
with open(hyper_jaccard_path, "w") as f:
    f.write("".join(file_lines))

In [4]:
file_lines = []
lines_buffer = []

with open(hyper_sketches_path, "r") as f:
    for line in f:
        if "{python_generated}" in line:
            # We flush the lines buffer.
            lines_buffer = []
            continue
        lines_buffer.append(line)
        if "{python_macro}" in line:
            if "{bits}" not in line or "{precision}" not in line or "{hops}" not in line:
                raise ValueError(
                    f"We detected a python macro line, namely {line}, "
                    "but it does not contain the expected tags {bits} and {precision} and {hops}."
                )
            # We split the line at the comments
            spacer = "//"
            if "///" in line:
                spacer = "///"
            initial_spaces, python_macro = line.split(spacer)
            for p in range(precision[0], precision[1] + 1):
                for b in range(bits[0], bits[1] + 1):
                    for h in range(hops[0], hops[1] + 1):
                        populated_line = initial_spaces + python_macro.strip().format(
                            bits=b,
                            precision=p,
                            hops=h,
                            python_macro="// {python_generated}\n"
                        )
                        lines_buffer.append(populated_line)
            file_lines.extend(lines_buffer)

file_lines.extend(lines_buffer)

with open(hyper_sketches_path, "w") as f:
    f.write("".join(file_lines))